In [1]:
# Standard imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

# Insert path to mavenn beginning of path
import os
import sys
abs_path_to_mavenn = os.path.abspath('../../')
sys.path.insert(0, abs_path_to_mavenn)

# Load mavenn
import mavenn
print(mavenn.__path__)

['/Users/jkinney/github/mavenn/mavenn']


In [2]:
# Load example data
data_df = mavenn.load_example_dataset('mpsa')
data_df.head()

,training_set,y,dy,x
0,False,-3.751854,0.444200,AAAGCAAAA
1,True,-2.697741,0.369972,AAAGCAAAC
2,True,-2.242947,0.575121,AAAGCAAAG
3,False,-3.067251,0.357014,AAAGCAAAT
4,False,-2.987074,0.472637,AAAGCAACA


In [3]:
# Split into trianing and test data
ix = data_df['training_set']
L = len(data_df['x'][0])
train_df = data_df[ix]
print(f'training N: {len(train_df):,}')
test_df = data_df[~ix]
print(f'testing N: {len(test_df):,}')

training N: 17,498
testing N: 4,431


In [4]:
model_kwargs = {
    'regression_type':'GE',
    'L':L,
    'alphabet':'dna',
    'ge_nonlinearity_type':'nonlinear',
    'gpmap_type':'mlp',
    'ge_noise_model_type':'SkewedT',
    'ge_heteroskedasticity_order':2
}

fit_kwargs={'learning_rate':.005,
            'epochs':100,
            'batch_size': 200,
            'early_stopping': True,
            'early_stopping_patience': 100,
            'linear_initialization': False}

file_name = 'mpsa_ge_blackbox'

In [5]:
if True:

    # Set seeed
    mavenn.set_seed(0)

    # Define model
    model = mavenn.Model(**model_kwargs)

    # Set training data
    model.set_data(x=train_df['x'],
                   y=train_df['y'])

    # Fit model to data
    model.fit(**fit_kwargs)
    
    # Save model
    model.save(file_name)

N = 17,498 observations set as training data.
Data shuffled.
Time to set data: 0.366 sec.
Epoch 1/100
70/70 [==============================] - 1s 8ms/step - loss: 250.9751 - I_var: -0.5099 - val_loss: 218.2813 - val_I_var: -0.3024
Epoch 2/100
70/70 [==============================] - 0s 3ms/step - loss: 195.1004 - I_var: -0.1216 - val_loss: 179.0225 - val_I_var: -0.0175
Epoch 3/100
70/70 [==============================] - 0s 3ms/step - loss: 168.9310 - I_var: 0.0680 - val_loss: 166.3812 - val_I_var: 0.0752
Epoch 4/100
70/70 [==============================] - 0s 3ms/step - loss: 159.2676 - I_var: 0.1386 - val_loss: 170.1410 - val_I_var: 0.0484
Epoch 5/100
70/70 [==============================] - 0s 3ms/step - loss: 154.9829 - I_var: 0.1703 - val_loss: 161.5241 - val_I_var: 0.1107
Epoch 6/100
70/70 [==============================] - 0s 3ms/step - loss: 154.7418 - I_var: 0.1730 - val_loss: 161.1336 - val_I_var: 0.1131
Epoch 7/100
70/70 [==============================] - 0s 2ms/step - loss:

In [6]:
model.layer_gpmap.input_shape

(None, 36)

In [7]:
model.get_theta().keys()

dict_keys(['L', 'C', 'alphabet', 'theta_0', 'theta_lc', 'theta_lclc', 'theta_mlp', 'logomaker_df'])

In [8]:
model.get_theta()['theta_mlp']

[[array([[ 1.58156380e-01, -9.93106589e-02, -8.18898901e-03,
           1.91798314e-01,  8.13778713e-02,  2.73752189e-03,
           9.29782242e-02,  1.95636362e-01,  1.92756012e-01,
          -2.63394924e-14],
         [ 8.24619159e-02, -4.97937910e-02,  5.57352193e-02,
          -1.61574967e-02,  1.82693928e-01, -9.38864127e-02,
          -7.76444376e-02,  1.11436360e-01,  5.56413457e-02,
          -1.05564558e-35],
         [-3.57295901e-01, -1.31085768e-01, -4.23249006e-02,
           1.28747839e-02,  4.84974273e-02,  2.35908836e-01,
           1.07815169e-01, -3.38197909e-02, -3.58532995e-01,
           3.24944029e-37],
         [-2.79000495e-02, -8.41639191e-02, -3.37245725e-02,
           1.60978828e-02, -8.68158400e-01, -8.56827758e-03,
           1.18786044e-01,  1.19688578e-01,  2.88222462e-01,
          -8.92286093e-36],
         [ 1.39934361e-01, -8.98744911e-03, -7.43143782e-02,
           5.84351793e-02,  5.93201339e-01, -1.52412683e-01,
          -2.39134610e-01,  1.0261

In [9]:
assert False

AssertionError: 

In [ ]:
# Load model
model = mavenn.load(file_name)

In [ ]:
# Get x and y
x_test = test_df['x'].values
y_test = test_df['y'].values
dy_test = test_df['dy'].values

In [ ]:
# Show training history
print('On test data:')

# Compute likelihood information
I_var, dI_var =  model.I_varlihood(x=x_test, y=y_test)
print(f'I_var_test: {I_var:.3f} +- {dI_var:.3f} bits') 

# Compute predictive information
I_pred, dI_pred = model.I_predictive(x=x_test, y=y_test)
print(f'I_pred_test: {I_pred:.3f} +- {dI_pred:.3f} bits')

# Compute intrinsic information
I_intr, dI_intr = mavenn.I_intrinsic(y_values=y_test, dy_values=dy_test)
print(f'I_intrinsic: {I_intr:.3f} +- {dI_intr:.3f} bits')

# Compute percent info explained
pct = 100*I_pred/I_intr
dpct = 100*np.sqrt((dI_pred/I_intr)**2 + (dI_intr*I_pred/I_intr**2)**2)
print(f'percent info explained: {pct:.1f}% +- {dpct:.1f}%')

I_var_hist = model.history['I_var']
val_I_var_hist = model.history['val_I_var']

fig, ax = plt.subplots(1,1,figsize=[4,4])
ax.plot(I_var_hist, label='I_var_train')
ax.plot(val_I_var_hist, label='I_var_val')
ax.axhline(I_var, color='C2', linestyle=':', label='I_var_test')
ax.axhline(I_pred, color='C3', linestyle=':', label='I_pred_test')
ax.axhline(I_intr, color='C4', linestyle=':', label='I_intrinsic')
ax.legend()
ax.set_xlabel('epochs')
ax.set_ylabel('bits')
ax.set_title('training hisotry')
ax.set_ylim([0, I_intr*1.2]);

In [ ]:
# Predict latent phentoype values (phi) on test data
phi_test = model.x_to_phi(x_test)

# Predict measurement values (yhat) on test data
yhat_test = model.x_to_yhat(x_test)

# Set phi lims and create grid in phi space
phi_lim = [min(phi_test)-.5, max(phi_test)+.5]
phi_grid = np.linspace(phi_lim[0], phi_lim[1], 1000)

# Compute yhat each phi gridpoint
yhat_grid = model.phi_to_yhat(phi_grid)

# Compute 90% CI for each yhat
q = [0.05, 0.95] #[0.16, 0.84]
yqs_grid = model.yhat_to_yq(yhat_grid, q=q)

# Create figure
fig, ax = plt.subplots(1, 1, figsize=[4, 4])

# Illustrate measurement process with GE curve
ax.scatter(phi_test, y_test, color='C0', s=5, alpha=.2, label='test data')
ax.plot(phi_grid, yhat_grid, linewidth=2, color='C1',
        label='$\hat{y} = g(\phi)$')
ax.plot(phi_grid, yqs_grid[:, 0], linestyle='--', color='C1', label='68% CI')
ax.plot(phi_grid, yqs_grid[:, 1], linestyle='--', color='C1')
ax.set_xlim(phi_lim)
ax.set_xlabel('latent phenotype ($\phi$)')
ax.set_ylabel('measurement ($y$)')
ax.set_title('measurement process')
ax.set_ylim([-6, 3])
ax.legend()

# Fix up plot
fig.tight_layout()
plt.show()

In [ ]:
# Plot pairwise parameters
if model.gpmap_type in ['pairwise', 'neighbor']:
    theta = model.get_theta()
    fig, ax = plt.subplots(1, 1, figsize=[8, 4])
    mavenn.heatmap_pairwise(values=theta['theta_lclc'],
                            alphabet=theta['alphabet'],
                            ax=ax);